<h1><a href="https://dl.acm.org/citation.cfm?id=2968498">
The SMT-Based Automatic Road Network Generation in Vehicle Simulation Environment</a></h1>
by BaekGyu Kim et al.


<h2>Summary</h2>

This paper uses SMT solver to generate curve segments that satisfy certain criteria. The curve segments can compose a road netowrk which can be used to test the correctness of vehicle control algorithms.

<h2>Background</h2>

* Defininng adequate coverage criteria and designing associated test case are two major considerations in tests. This paper focuses on test case generation.

* In vehicle simulation, the test cases are various road envioronment. The plain and straight forward solution is to virtually simulate real road scenario. However, it is hard to create road structures that satisfy specified test coverage criteria.

<h2>Problem Formulation</h2>

* `Interpolation` is used in this paper to generate road structures based on sets of 3D coordinates.
    * A set of 3D coordinates can be interpolated in a spline and results in a set of curve segments.
    * Each curve is segmented with a pair of 3D coordinates as its end points and two consecutive curves share one same end point.
    * Each 3D coordinate point can be shared at most by two consecutive curves.
    * An interpolation can be used as a centreline of a road. With additional parameters, a virtual road can be simulated.

* `Curve Coverage Criteria` is a tuple $(N, \Theta, D)$, where $N$ is the min/max number of curves in a spline/globally in all the splines; $\Theta$ is the min/max curvature of each curve segment $[\theta_{min}, \theta_{max}]$; $D$ is the min/max distance of any two adjacent curves $[d_{min}, d_{max}]$.
    * **With a given curve coverage criteria, the problem is to find a set of coordinates of which the interpolation satisfies the curve coverage criteria.**
    

<h2>Approach</h2>

<h3>3D coordinates generation</h3>

Assuming that a satisfying set of 3D coordinates is $\{(x_k^i, y_k^i, z_k^i)\}^{k=n}_{k=1}$ and its interpolation composes a road segment $r_i$, to formulate the problem of generating this set of 3D coordinates as a SMT problem, constraints must be extracted from the curve coverage criteria. 

* Boundary constraints: the coordinates should be bounded in a space, e.g. for a box, the constratints are like
$$(x_{min}\leq x_i \leq x_{max})\wedge (y_{min}\leq y_i \leq y_{max}) \wedge (z_{min}\leq z_i \leq z_{max})$$

* Curve number constraint: the number of the curve, or the number of the 3D coordinates in another word, should be bounded in a range $[n_{min}, n_{max}]$.

* Curve distance constraint: the distance between the end points for each curve is bounded, e.g.
$$d_{min}\leq\sqrt{(x_{k+1}-x_k)^2+(y_{k+1}-y_k)^2+(z_{k+1}-z_k)^2}\leq d_{max}$$

* Curvature constraint: the curvature between each pair of consecutive curves is bounded, e.g.
$$\theta_{min}\leq(|\frac{y_{k+1} - y_k}{x_{k+1}-x_k}-\frac{y_{k+2} - y_{k+1}}{x_{k+2}-x_{k+1}}|)\leq \theta_{max}$$
$$\theta_{min}\leq(|\frac{z_{k+1} - z_k}{x_{k+1}-x_k}-\frac{z_{k+2} - z_{k+1}}{x_{k+2}-x_{k+1}}|)\leq \theta_{max}$$

* Althernating road constraint: the 'directions' between consecutive curves should be opposite such that the segmentation is necessary, e.g.
$$(\frac{y_{k+1} - y_k}{x_{k+1}-x_k})\cdot(\frac{y_{k+2} - y_{k+1}}{x_{k+2}-x_{k+1}})\leq 0$$

The algorithm proposed in the paper generates road segments iteratively. Every time when a statisfying road segment is returned by the SMT solver, the road segment is added to the solution set and an additional constraint is created and added to the SMT fomula to prohibit the solver from repeatedly generating solutions that are same as the existing ones.

<img src='./fig1.png'></img>
<img src='./fig2.png'></img>

<h4>Sequential Road Network Generation</h4>

To generate road network, not only the local curve coverage criterias for individual road segments should be satisfied, but also should the global curve coverage criteria $C_g$ be satisfied if it exists, e.g. the curvature between joint road segments should be bounded.

The method the paper adopts is to check the consistency between local criteria and global criteria in the very beginning. For instance, if there are $k$ local curve coverage criteria $[C_i]^k_1$ and for each $C_i^k$ the number of curves is constrined by $[C_i^k.n_{min}, C_i^k.n_{max}]$, then $((k-1)+\sum^{i=k}_{i=1}C_i.n_{min}\geq C_g.n_{min})$ should be satisfied where $k-1$ indicates that there are $k-1$ joints between the road segments generated for each local criteria. Constraints like this are sufficient but not necessary preconditions for the existence of the road network.

If all the preconditions are satisfied, the generation of the road network begins. For each local criteria, one set of curve segment is generated. Similar to 3D coordinate generation, after a local criteria is take care of, new additional constraints are created such that 
* the end point of the most recently created road segment is the beginning point of the next road segment to be solved, i.e. road segments should be connected consecutively.
* the curvature between the end curve of the latest created road segments and the beginning curve of the road segment to be solved next, i.e. the curvature between joint road segments, should satisfy the global criteria.







<h2>Opinion</h2>

In short, this paper transform coverage criterias of test cases into SMT formulas and uses the solved 3D coordinates to generate road segments. 

One realistic problem is how to defind the coverage criterias which, whereas has been mentioned in the beginning of the paper, cannot be neglected. If we are to design a system that can generate test cases that can lead to falsification，we can 

* generate test cases directly based on the specification

* or generate test cases based on cover criterias, then
** if test cases can be generated in the same way as in this paper, then cover criterias must be determined in some subtle way so that they are consistent with the specification.
** if we are to define the cover criterias in a different way, then a new and different method is also needed to generate test cases from the criterias.

